<a href="https://colab.research.google.com/github/kyochanpy/Google_Smartphone_Decimeter_Challenge/blob/main/filter/kalman_filter_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from pathlib import Path 
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [17]:
# data
path = Path("/content/drive/MyDrive/GSDC")
test_base = pd.read_csv(path / "baseline_locations_test.csv")
sub = pd.read_csv(path / "sample_submission.csv")

truths = (path / "train").rglob("ground_truth.csv")

df_list = []
cols = ["collectionName", "phoneName", "millisSinceGpsEpoch", "latDeg", "lngDeg"]

for t in tqdm(truths, total=73):
    df_phone = pd.read_csv(t, usecols=cols)
    df_list.append(df_phone)
df_truth = pd.concat(df_list, ignore_index=True)

train_base = pd.read_csv(path / "baseline_locations_train.csv", usecols=cols)
all_df = df_truth.merge(train_base, how="inner", on=cols[:3], suffixes=("_truth", '_train_base'))

In [18]:
def kalman_filter(input_df):
    !pip install simdkalman
    import simdkalman

    T = 1.0
    state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                                [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
    process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
    observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
    observation_noise = np.diag([1e-4, 1e-4]) + np.ones((2, 2)) * 1e-9

    kf = simdkalman.KalmanFilter(
            state_transition = state_transition,
            process_noise = process_noise,
            observation_model = observation_model,
            observation_noise = observation_noise)
    
    def apply_kf_smoothing(df, kf_=kf):
        unique_paths = df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
        for collection, phone in tqdm(unique_paths):
            cond = np.logical_and(df['collectionName'] == collection, df['phoneName'] == phone)
            data = df[cond][['latDeg', 'lngDeg']].to_numpy()
            data = data.reshape(1, len(data), 2)
            smoothed = kf_.smooth(data)
            df.loc[cond, 'latDeg'] = smoothed.states.mean[0, :, 0]
            df.loc[cond, 'lngDeg'] = smoothed.states.mean[0, :, 1]
        return df

    kf_smoothed_baseline = apply_kf_smoothing(input_df)
    output_df = sub.assign(latDeg = kf_smoothed_baseline.latDeg, lngDeg = kf_smoothed_baseline.lngDeg)
    
    return output_df

In [19]:
a = kalman_filter(test_base)